In [38]:
import os
import numpy as np
import h5py
import torch
cuda = torch.cuda.is_available()

### Import file

In [36]:
FILE_DIR = 'C:\Big_Data_Files\G2.h5'
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
FILE_DIR

'C:\\Big_Data_Files\\G2.h5'

In [12]:
f = h5py.File(FILE_DIR, 'r')
list(f.keys())

['b_', 'ih', 'ih_mean']

In [34]:
data = f['ih'][...]
data = torch.from_numpy(data)

In [59]:
print(data.size())

torch.Size([78979, 3, 128, 128])


In [48]:
batch_size = 16
validation_split = .1
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices, val_indices = indices[split+split:], indices[split:split+split], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(data, batch_size=batch_size,
                          sampler=train_sampler, pin_memory=cuda)
valid_loader  = DataLoader(data, batch_size=batch_size, 
                          sampler=valid_sampler, pin_memory=cuda)
test_loader  = DataLoader(data, batch_size=batch_size, 
                          sampler=test_sampler, pin_memory=cuda)


### Code